In [1]:
import os, sys
import jieba, codecs, math
import jieba.posseg as pseg
import pickle

In [2]:
names = {}
relationships = {}
lineNames = []

path = '/dataset/SNA_final/'
_01file = path+"三國001.txt"
cut_dict = path+"all_dict.txt"

In [3]:
all_name = pickle.load(open(path+'all_name.pkl', 'rb'))
all_2ndname_dict = pickle.load(open(path+'all_2ndname.pkl', 'rb'))

In [4]:
def check_the_word(word):
    all_2ndname = list(all_2ndname_dict.values())
    
    if word in all_2ndname:
        word = list(all_2ndname_dict.keys())[list(all_2ndname_dict.values()).index(word)]
        
    return word

In [5]:
# load dictionary
jieba.load_userdict(cut_dict)

with codecs.open(_01file.encode('utf-8'), "r", "utf8") as f:
    for line in f.readlines():
        
        # cut the word and return the part of speech to the list
        # ex: [pair('第', 'm'), pair('001', 'm'), pair('回', 'v')...]
        poss = pseg.cut(line)
        lineNames.append([])
        for w in poss:
            # to determine whether it is a person's name
            if w.flag != "nr" or len(w.word) < 2:
                continue
                
            organized_word = check_the_word(w.word)
            
            lineNames[-1].append(organized_word)
            if names.get(organized_word) is None:
                names[organized_word] = 0
                relationships[organized_word] = {}
            names[organized_word] += 1

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.069 seconds.
Prefix dict has been built succesfully.


In [6]:
for name, times in names.items():
    print(name, times)

張縱馬 1
曉夜 1
曹騰 1
侍奉 1
蹇碩 2
廣宗來 1
玄德回 1
溫德殿 2
寇發 1
張名飛 1
左有 1
花開正 1
關羽 9
阿父 1
劉名備 1
盧中郎 1
漢魯恭 1
郎蔡邕 1
鄭玄 1
於桓 1
曹操 1
漢武 1
馬引軍 1
玄德甚 1
玄德大 1
玄德兵 1
郭勝 1
陳蕃 2
王之後 1
張寶 8
曹嵩 3
巨鹿郡 1
劉焉 9
劉貞 1
馬匹 1
唐周 2
大賢良 2
夏惲 1
莊田 1
玄孫 1
若萌 1
桓帝 2
張讓 2
童童 1
高祖 1
天公 2
張引 1
玄德引 3
黃巾蓋 1
馬元義 2
董卓 5
許劭 1
蒼天 1
公看榜而 1
何進 2
漢景帝 1
劉雄 1
夏侯氏 1
金失侯 1
於玉堂 1
鄧茂 3
廣宗 3
明公宜 1
於園 1
束草 1
聞中郎 1
童顏 1
桑村 1
蘇雙 1
謝別二客 1
鄒靖 8
程遠志 5
於桃園 1
侯覽 1
干政之 1
秦滅 1
汝可引 1
張來 1
齊聲 1
安民 2
帝尊信 1
關公 1
劉弘 1
丹鳳 1
張世平 1
張天 1
梁上飛 1
都尉 2
立功 2
黃巾 3
張角 13
靖王 1
關名羽 1
帝覽 1
盧植 10
曹節 3
靈帝 1
左豐 2
張飛 12
劉備 34
玄德遂 1
龔景 3
汝中風 1
大吉 1
若巨雷 1
張梁 8
竇武 2
玄德曰 8
張曰 1
南華老仙 1
武中興 1
弘曾 1
公孫瓚 1
劉勝 2
皇甫嵩 5
汝得之 1
桑樹 1
山后 1
程曠 1
趙忠 1
劉元起 1


In [7]:
# for each segment
for line in lineNames:                    
    for name1 in line:   
        # any two people in each segment
        for name2 in line:  
            if name1 == name2:
                continue
            
            # create new items if they haven’t appeared at the same time
            if relationships[name1].get(name2) is None:        
                relationships[name1][name2]= 1
            else:
                # the number of co-occurrences of two people plus 1
                relationships[name1][name2] = relationships[name1][name2] + 1        

In [8]:
with codecs.open(path+"Three_Kingdoms_node.txt", "w", "utf8") as f:
    f.write("Id Label Weight\r\n")
    for name, times in names.items():
        f.write(name + " " + name + " " + str(times) + "\r\n")
        
with codecs.open(path+"Three_Kingdoms_edge.txt", "w", "utf8") as f:
    f.write("Source Target Weight\r\n")
    for name, edges in relationships.items():
        for v, w in edges.items():
            if w > 3:
                f.write(name + " " + v + " " + str(w) + "\r\n")